In [ ]:
!pip install qiskit
from qiskit import *
!pip install qiskit-aer
!pip install qiskit-optimization
!pip install qiskit_algorithms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.6/635.6 kB 47.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created 

In [ ]:
import qiskit_algorithms
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import *
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np
import math

In [ ]:
def get_filtered_samples(
    samples: List[SolutionSample],
    threshold: float = 0,
    allowed_status: Tuple[OptimizationResultStatus] = (OptimizationResultStatus.SUCCESS,),
):
    res = []
    for s in samples:
        if s.status in allowed_status and s.probability > threshold:
            res.append(s)

    return res

In [ ]:
def generate_top_down(side_length, way, desired, combos):
  qubo = QuadraticProgram()
  #initializing variables
  for i in range(side_length):
      for k in combos:
        qubo.binary_var(f"{i}{k}")

  #number of options
  linear = [1] * side_length * len(combos)
  qubo.minimize(linear=linear, quadratic={})

  constraints = {}
  constraintRand = {}
  constraintConstrained = {}
  total = 0
  curr = 1

  #only one in each box
  for k in range(side_length):
    total += curr
    for i in range(len(combos)):
      constraints[f"{k}{combos[i]}"] = curr
      constraintRand[f"{k}{combos[i]}"] = 0
      constraintConstrained[f"{k}{combos[i]}"] = 0
    curr *= 10

  # so heres how we do this, we pretend its all right to left with up and down as top contrained

  #Right and left must be aligned
  #Way your not looking at
  curr = 1
  for i in range(side_length):
    if i == 0: #if we're on one side of the end we only looks at the right value
      curr *= 10
      for combo in combos: #for each combo
        constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[1]) + 1) * curr)
    elif i  == (side_length - 1): #if we're on other side of end
        for combo in combos:
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[3]) + 1) * curr) * -1 #only looks at the left value
    else: #if we're in middle
        for combo in combos:
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[3]) + 1) * curr) * -1
        curr *= 10 #we multiply in the middle since we want the left to be same as previous right
        for combo in combos:
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[1]) + 1) * curr)
    qubo.linear_constraint(linear=constraintRand, sense="==", rhs=0)

    curr = 1
    #top and bottom must be aligned correctly
    for i in range(2):
        for combo in combos: #for each combo
          constraintConstrained[f"block{i}"] = (desired[i] + 1) * -1 * curr
          constraintConstrained[f"{i}{combo}"] = (desired[i] + 1) * curr
        curr *= 10
    qubo.linear_constraint(linear=constraintConstrained, sense="==", rhs=0)

    #quanutm stuff
  converter = QuadraticProgramToQubo()
  qubo_with_penalty = converter.convert(qubo)
  op, offset = qubo_with_penalty.to_ising()
  qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
  qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
  qaoa_result = qaoa.solve(qubo)

  filtered_samples = get_filtered_samples(
    qaoa_result.samples, threshold=0.00001, allowed_status=(OptimizationResultStatus.SUCCESS,)
  )

  samples_for_plot = {}
  for s in filtered_samples:
        x = 0
        sample_str = " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x))

        samples_for_plot[sample_str] = s.probability
  length = len(samples_for_plot)
  samples_for_plot = {}
  w, h = side_length, length #h is number probailitues
  arr = [[0 for x in range(w)] for y in range(h)]
  l = 0

  for s in filtered_samples:
      x = 0
      sample_str = " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x))

      samples_for_plot[sample_str] = s.probability
      for i, v in enumerate(s.x):
          if int(v) == 1 and x < 2:
              arr[l][x] = qaoa_result.variables[i].name[1:]
              x+=1
      l += 1
  return arr[0]

In [ ]:
def generate_side(side_length, way, desired, combos):
  qubo = QuadraticProgram()
  #initializing variables
  for i in range(side_length):
      for k in combos:
        qubo.binary_var(f"{i}{k}")

  #number of options
  linear = [1] * side_length * len(combos)
  qubo.minimize(linear=linear, quadratic={})

  constraints = {}
  constraintRand = {}
  constraintConstrained = {}
  total = 0
  curr = 1

  #only one in each box
  for k in range(side_length):
    total += curr
    for i in range(len(combos)):
      constraints[f"{k}{combos[i]}"] = curr
      constraintRand[f"{k}{combos[i]}"] = 0
      constraintConstrained[f"{k}{combos[i]}"] = 0
    curr *= 10

  # so heres how we do this, we pretend its all right to left with up and down as top contrained

  #Way your not looking at
  if (way % 2 == 0):
    curr = 1
    for i in range(side_length):
      if i == 0: #if we're on one side of the end we only looks at the right value
        curr *= 10
        for combo in combos: #for each combo
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[1]) + 1) * curr)
      elif i  == (side_length - 1): #if we're on other side of end
          for combo in combos:
            constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[3]) + 1) * curr) * -1 #only looks at the left value
      else: #if we're in middle
          for combo in combos:
            constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[3]) + 1) * curr) * -1
          curr *= 10 #we multiply in the middle since we want the left to be same as previous right
          for combo in combos:
            constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[1]) + 1) * curr)
      qubo.linear_constraint(linear=constraintRand, sense="==", rhs=0)

      curr = 1
      #top and bottom must be aligned correctly
      for i in range(2):
          for combo in combos: #for each combo
            constraintConstrained[f"block{i}"] = (desired[i] + 1) * -1 * curr
            constraintConstrained[f"{i}{combo}"] = (desired[i] + 1) * curr
          curr *= 10
      qubo.linear_constraint(linear=constraintConstrained, sense="==", rhs=0)
    else:
      for i in range(side_length):
        for combo in combos: #for each combo
          constraintConstrained[f"block{i}"] = (desired[i] + 1) * -1 * curr
          constraintConstrained[f"{i}{combo}"] = (desired[i] + 1) * curr
        curr *= 10
      qubo.linear_constraint(linear=constraintConstrained, sense="==", rhs=0)

      #Right and left must be aligned
      #Way your not looking at
      curr = 1
      for i in range(side_length):
        if i == 0: #if we're on one side of the end we only looks at the right value
          curr *= 10
          for combo in combos: #for each combo
            constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[2]) + 1) * curr)
        elif i  == (side_length - 1): #if we're on other side of end
            for combo in combos:
              constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[0]) + 1) * curr) * -1 #only looks at the left value
        else: #if we're in middle
            for combo in combos:
              constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[0]) + 1) * curr) * -1
            curr *= 10 #we multiply in the middle since we want the left to be same as previous right
            for combo in combos:
              constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[2]) + 1) * curr)
        qubo.linear_constraint(linear=constraintRand, sense="==", rhs=0)

    #quanutm stuff
  converter = QuadraticProgramToQubo()
  qubo_with_penalty = converter.convert(qubo)
  op, offset = qubo_with_penalty.to_ising()
  qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
  qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
  qaoa_result = qaoa.solve(qubo)

  filtered_samples = get_filtered_samples(
    qaoa_result.samples, threshold=0.00001, allowed_status=(OptimizationResultStatus.SUCCESS,)
  )

  samples_for_plot = {}
  for s in filtered_samples:
        x = 0
        sample_str = " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x))

        samples_for_plot[sample_str] = s.probability
  length = len(samples_for_plot)
  samples_for_plot = {}
  w, h = side_length, length #h is number probailitues
  arr = [[0 for x in range(w)] for y in range(h)]
  l = 0

  for s in filtered_samples:
      x = 0
      sample_str = " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x))

      samples_for_plot[sample_str] = s.probability
      for i, v in enumerate(s.x):
          if int(v) == 1 and x < 2:
              arr[l][x] = qaoa_result.variables[i].name[1:]
              x+=1
      l += 1
  return arr[0]

In [ ]:
def generate_right_left(side_length, way, desired, combos):
  qubo = QuadraticProgram()
  #initializing variables
  for i in range(side_length):
      for k in combos:
        qubo.binary_var(f"{i}{k}")

  #number of options
  linear = [1] * side_length * len(combos)
  qubo.minimize(linear=linear, quadratic={})

  constraints = {}
  constraintRand = {}
  constraintConstrained = {}
  total = 0
  curr = 1

  #only one in each box
  for k in range(side_length):
    total += curr
    for i in range(len(combos)):
      constraints[f"{k}{combos[i]}"] = curr
      constraintRand[f"{k}{combos[i]}"] = 0
      constraintConstrained[f"{k}{combos[i]}"] = 0
    curr *= 10

  # so heres how we do this, we pretend its all right to left with up and down as top contrained


  for i in range(side_length):
        for combo in combos: #for each combo
          constraintConstrained[f"block{i}"] = (desired[i] + 1) * -1 * curr
          constraintConstrained[f"{i}{combo}"] = (desired[i] + 1) * curr
        curr *= 10
  qubo.linear_constraint(linear=constraintConstrained, sense="==", rhs=0)

  #Right and left must be aligned
  #Way your not looking at
  curr = 1
  for i in range(side_length):
    if i == 0: #if we're on one side of the end we only looks at the right value
      curr *= 10
      for combo in combos: #for each combo
        constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[2]) + 1) * curr)
    elif i  == (side_length - 1): #if we're on other side of end
        for combo in combos:
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[0]) + 1) * curr) * -1 #only looks at the left value
    else: #if we're in middle
        for combo in combos:
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[0]) + 1) * curr) * -1
        curr *= 10 #we multiply in the middle since we want the left to be same as previous right
        for combo in combos:
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[2]) + 1) * curr)
    qubo.linear_constraint(linear=constraintRand, sense="==", rhs=0)

    #quanutm stuff
  converter = QuadraticProgramToQubo()
  qubo_with_penalty = converter.convert(qubo)
  op, offset = qubo_with_penalty.to_ising()
  qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
  qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
  qaoa_result = qaoa.solve(qubo)

  filtered_samples = get_filtered_samples(
    qaoa_result.samples, threshold=0.00001, allowed_status=(OptimizationResultStatus.SUCCESS,)
  )

  samples_for_plot = {}
  for s in filtered_samples:
        x = 0
        sample_str = " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x))

        samples_for_plot[sample_str] = s.probability
  length = len(samples_for_plot)
  samples_for_plot = {}
  w, h = side_length, length #h is number probailitues
  arr = [[0 for x in range(w)] for y in range(h)]
  l = 0

  for s in filtered_samples:
      x = 0
      sample_str = " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x))

      samples_for_plot[sample_str] = s.probability
      for i, v in enumerate(s.x):
          if int(v) == 1 and x < 2:
              arr[l][x] = qaoa_result.variables[i].name[1:]
              x+=1
      l += 1
  return arr[0]

In [ ]:
def generate_base(combos, shortest_side):
  if shortest_side % 2 == 0:
    side_length = 2
  else:
    side_length = 1

  qubo = QuadraticProgram()
  #initializing variables
  for i in range(side_length):
      for k in combos:
        qubo.binary_var(f"{i}{k}")

  #number of options
  linear = [1] * side_length * len(combos)
  qubo.minimize(linear=linear, quadratic={})

  constraints = {}
  constraintRand = {}
  constraintConstrained = {}
  total = 0
  curr = 1

  #only one in each box
  for k in range(side_length):
    total += curr
    for i in range(len(combos)):
      constraints[f"{k}{combos[i]}"] = curr
      constraintRand[f"{k}{combos[i]}"] = 0
      constraintConstrained[f"{k}{combos[i]}"] = 0
    curr *= 10

  # so heres how we do this, we pretend its all right to left with up and down as top contrained

  #Right and left must be aligned
  #Way your not looking at
  curr = 1
  for i in range(side_length):
    if i == 0: #if we're on one side of the end we only looks at the right value
      curr *= 10
      for combo in combos: #for each combo
        constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[1]) + 1) * curr)
    elif i  == (side_length - 1): #if we're on other side of end
        for combo in combos:
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[3]) + 1) * curr) * -1 #only looks at the left value
    else: #if we're in middle
        for combo in combos:
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[3]) + 1) * curr) * -1
        curr *= 10 #we multiply in the middle since we want the left to be same as previous right
        for combo in combos:
          constraintRand[f"{i}{combo}"] = constraintRand[f"{i}{combo}"] + ((int(combo[1]) + 1) * curr)
    qubo.linear_constraint(linear=constraintRand, sense="==", rhs=0)

    #quanutm stuff
  converter = QuadraticProgramToQubo()
  qubo_with_penalty = converter.convert(qubo)
  op, offset = qubo_with_penalty.to_ising()
  qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
  qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
  qaoa_result = qaoa.solve(qubo)

  filtered_samples = get_filtered_samples(
    qaoa_result.samples, threshold=0.00001, allowed_status=(OptimizationResultStatus.SUCCESS,)
  )

  samples_for_plot = {}
  for s in filtered_samples:
        x = 0
        sample_str = " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x))

        samples_for_plot[sample_str] = s.probability
  length = len(samples_for_plot)
  samples_for_plot = {}
  w, h = side_length, length #h is number probailitues
  arr = [[0 for x in range(w)] for y in range(h)]
  l = 0

  for s in filtered_samples:
      x = 0
      sample_str = " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x))

      samples_for_plot[sample_str] = s.probability
      for i, v in enumerate(s.x):
          if int(v) == 1 and x < 2:
              arr[l][x] = qaoa_result.variables[i].name[1:]
              x+=1
      l += 1
  return arr[0]


In [ ]:
def generate_corners(combos, way1, way2, constrained1, constrained2):
  qubo = QuadraticProgram()
  #initializing variables
  for i in range(1):
      for k in combos:
        qubo.binary_var(f"{i}{k}")

  #number of options
  linear = [1] * len(combos)
  qubo.minimize(linear=linear, quadratic={})
  constraints = {}
  constraintConstrained = {}
  total = 0
  curr = 1

  #only one in each box
  for k in range(1):
    total += curr
    for i in range(len(combos)):
      constraints[f"{k}{combos[i]}"] = curr
      constraintConstrained[f"{k}{combos[i]}"] = 0
    curr *= 10

  curr = 1

  #Right and left
  for combo in combos: #for each combo
      constraintConstrained[f"{1}{combo}"] = (int(combo[way1]) + 1) * curr
      constraintConstrained[f"{1}{combo}"] = (int(combo[way2]) + 1) * curr * 10

  constraintConstrained["block1"] = (constrained1 + 1) * -1
  constraintConstrained["block2"] = (constrained2 + 1) * -10
  qubo.linear_constraint(linear=constraintConstrained, sense="==", rhs=0)


In [ ]:

combos = ['0000','0110','1111', '0101', '1010']
x = generate_base(combos, 4)
print(x)
generate_top_down(2,0,x,combos)

[0, 0]


KeyError: 'block0'